In [1]:
!pip install neo4j

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.8.0-py3-none-any.whl size=258422 sha256=040e9c59618f8c4c586d3ef619247714d160651fd5662877d3d8dfda39732c9e
  Stored in directory: c:\users\sanmeet\appdata\local\pip\cache\wheels\3e\ef\4c\a91599f59166356b3942580360be2362563a7a97c877d018ca
Successfully built neo4j


In [1]:
from neo4j import __version__ as neo4j_version
from pandas import DataFrame
print(neo4j_version)

5.8.0


In [2]:
from neo4j import GraphDatabase

In [3]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
# create an instance of connection with the parameters
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="metaphorkb2")

In [5]:
query_string = '''
MATCH (s:SOURCE)-[attr:ATTRIBUTE]->(t:TARGET)
WHERE s.value =~ '.*star.*'
RETURN t.value, attr.value, s.value
'''
dtf_data = DataFrame([dict(_) for _ in conn.query(query_string, db='neo4j')])
dtf_data

,t.value,attr.value,s.value
0,idea,bright,shining star
1,opportunity,fleeting,shooting star
2,passing,fast,shooting star
3,aeroplane,quickly,shooting star
4,ardent face,out looking,star
5,engagement ring,sparkle,star
6,she,radiant,star
7,carrier,continuous,stars
8,faces,pale,evening stars
9,eyes,bright,blazing star


In [6]:
target = 'eye'
source = 'diamond'

In [8]:
query_string = '''
MATCH (s:SOURCE)-[attr:ATTRIBUTE]->(t:TARGET)
WHERE t.value =~ '.*{target}.*' AND s.value =~ '.*{source}.*'
RETURN t.value, attr.value, s.value
'''.format(target=target, source=source)

dtf_data = DataFrame([dict(_) for _ in conn.query(query_string, db='neo4j')])
possible_attributes = set(dtf_data["attr.value"])

print("Possible Attributes for Target: {target} and Source: {source} are:\n".format(target=target, source=source))
lst = list(possible_attributes)
for i in range(len(lst)):
    print(lst[i])

Possible Attributes for Target: eye and Source: diamond are:

gleam
sparkle


In [11]:
conn.close()